In [1]:
# Packages
import os
import yfinance as yf
from haystack.nodes import LinkContentFetcher
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import  BM25Retriever, AnswerParser
from haystack.nodes.prompt import PromptTemplate
from haystack.nodes import PromptNode
from haystack import Pipeline
from haystack.utils import print_answers

In [2]:
# Model configurations
hfAPIKey = os.getenv("HF_API_KEY")
hfModelName = os.getenv("HF_MODEL_NAME")

In [3]:
gold = yf.Ticker("GC=F")
news = gold.news

In [4]:
links = []

for new in news:
    links.append(new["link"])

In [5]:
lcf = LinkContentFetcher()
docs = []

for link in links:
    docs.append(lcf.fetch(link)[0])

In [6]:
documentStore = InMemoryDocumentStore(use_bm25=True)

documentStore.delete_documents()
documentStore.write_documents(docs)

retriever = BM25Retriever(document_store=documentStore, top_k=3)



Updating BM25 representation...: 100%|██████████| 8/8 [00:00<00:00, 1716.60 docs/s]


In [7]:
promptTemplate = PromptTemplate(
    prompt="""
    Answer the question truthfully based solely on the given documents. If the documents do not contain the answer to the question, say that answering is not possible given the available information. Your answer should be no longer than 50 words.
    Documents:{join(documents)}
    Question:{query}
    Answer:
    """,
    output_parser=AnswerParser(),
)

# Prompt Node initialization
promptNode = PromptNode(hfModelName,api_key=hfAPIKey,default_prompt_template = promptTemplate)

In [8]:
# Pipeline
generative_pipeline = Pipeline()
generative_pipeline.add_node(component=retriever, name="retriever", inputs=["Query"])
generative_pipeline.add_node(component=promptNode, name="promptNode", inputs=["retriever"])

In [10]:
response = generative_pipeline.run("How did gold perform?")
print_answers(response, details="minimum")

The prompt has been truncated from 6588 tokens to 924 tokens so that the prompt length and answer length (100 tokens) fit within the max token limit (1024 tokens). Shorten the prompt to prevent it from being cut off.


'Query: How did gold perform?'
'Answers:'
[   {   'answer': ' from sanctions by the Biden administration should be '
                  'frozen.\n'
                  'The administration has already been working to do that.\n'
                  'The US Treasury Department has been working to freeze $6 '
                  'billion in Iranian oil assets that were freed from '
                  'sanctions by the Biden administration.\n'
                  'The Treasury Department has been working to freeze $6 '
                  'billion in Iranian oil assets that were freed from '
                  'sanctions by the Biden administration.\n'
                  'The Treasury Department has been working to freeze $6 '
                  'billion in Iranian oil assets that'}]
